In [6]:
import pytorch_tabular
import pytorch_tabular.models

In [9]:
print(pytorch_tabular.__version__)

1.1.0


In [11]:
import torch
print(torch.__version__)

2.3.1+cpu


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Muat dataset Iris
iris = load_iris()
X = iris.data
y = iris.target

# Normalisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Bagi data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Persiapkan data untuk DataLoader
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion),
            nn.ReLU(),
            nn.Linear(forward_expansion, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(query, key, value, attn_mask=mask)[0]
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

class TabularTransformer(nn.Module):
    def __init__(self, num_features, num_classes, embed_size, num_heads, forward_expansion, dropout):
        super(TabularTransformer, self).__init__()
        self.embedding = nn.Linear(num_features, embed_size)
        self.transformer_block = TransformerBlock(
            embed_size=embed_size,
            heads=num_heads,
            dropout=dropout,
            forward_expansion=forward_expansion
        )
        self.fc = nn.Linear(embed_size, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add sequence dimension
        x = self.embedding(x)
        x = self.transformer_block(x, x, x, None)
        x = x.squeeze(1)  # Remove sequence dimension
        out = self.fc(x)
        return out

# Model
model = TabularTransformer(
    num_features=X.shape[1],
    num_classes=len(set(y)),
    embed_size=32,
    num_heads=4,
    forward_expansion=64,
    dropout=0.1
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f'Epoch {epoch+1}/{num_epochs}, Accuracy: {correct / total:.4f}')

Epoch 1/100, Accuracy: 0.2667
Epoch 2/100, Accuracy: 0.4667
Epoch 3/100, Accuracy: 0.8000
Epoch 4/100, Accuracy: 0.9333
Epoch 5/100, Accuracy: 0.9333
Epoch 6/100, Accuracy: 0.9333
Epoch 7/100, Accuracy: 0.9333
Epoch 8/100, Accuracy: 0.9667
Epoch 9/100, Accuracy: 0.9667
Epoch 10/100, Accuracy: 0.9667
Epoch 11/100, Accuracy: 0.9667
Epoch 12/100, Accuracy: 0.9667
Epoch 13/100, Accuracy: 0.9667
Epoch 14/100, Accuracy: 0.9667
Epoch 15/100, Accuracy: 1.0000
Epoch 16/100, Accuracy: 1.0000
Epoch 17/100, Accuracy: 1.0000
Epoch 18/100, Accuracy: 0.9667
Epoch 19/100, Accuracy: 0.9667
Epoch 20/100, Accuracy: 0.9667
Epoch 21/100, Accuracy: 0.9667
Epoch 22/100, Accuracy: 0.9667
Epoch 23/100, Accuracy: 1.0000
Epoch 24/100, Accuracy: 1.0000
Epoch 25/100, Accuracy: 0.9667
Epoch 26/100, Accuracy: 0.9667
Epoch 27/100, Accuracy: 1.0000
Epoch 28/100, Accuracy: 1.0000
Epoch 29/100, Accuracy: 0.9667
Epoch 30/100, Accuracy: 0.9667
Epoch 31/100, Accuracy: 0.9667
Epoch 32/100, Accuracy: 1.0000
Epoch 33/100, Acc

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Muat dataset Iris
iris = load_iris()
X = iris.data
y = iris.target

# Normalisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Bagi data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Persiapkan data untuk DataLoader
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

class SAINT(nn.Module):
    def __init__(self, num_features, num_classes, embed_size, num_heads, num_layers, forward_expansion, dropout):
        super(SAINT, self).__init__()
        self.embedding = nn.Linear(num_features, embed_size)
        self.transformer_blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=embed_size,
                nhead=num_heads,
                dim_feedforward=forward_expansion,
                dropout=dropout
            ) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(embed_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add sequence dimension
        for transformer in self.transformer_blocks:
            x = transformer(x)
        x = x.squeeze(1)  # Remove sequence dimension
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Model
model = SAINT(
    num_features=X.shape[1],
    num_classes=len(set(y)),
    embed_size=32,
    num_heads=4,
    num_layers=2,
    forward_expansion=64,
    dropout=0.1
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f'Epoch {epoch+1}/{num_epochs}, Accuracy: {correct / total:.4f}')

Epoch 1/100, Accuracy: 0.3000
Epoch 2/100, Accuracy: 0.9667
Epoch 3/100, Accuracy: 0.9333
Epoch 4/100, Accuracy: 0.9333
Epoch 5/100, Accuracy: 0.9333
Epoch 6/100, Accuracy: 0.9333
Epoch 7/100, Accuracy: 0.9667
Epoch 8/100, Accuracy: 1.0000
Epoch 9/100, Accuracy: 1.0000
Epoch 10/100, Accuracy: 1.0000
Epoch 11/100, Accuracy: 1.0000
Epoch 12/100, Accuracy: 1.0000
Epoch 13/100, Accuracy: 0.9667
Epoch 14/100, Accuracy: 0.9667
Epoch 15/100, Accuracy: 1.0000
Epoch 16/100, Accuracy: 1.0000
Epoch 17/100, Accuracy: 1.0000
Epoch 18/100, Accuracy: 1.0000
Epoch 19/100, Accuracy: 1.0000
Epoch 20/100, Accuracy: 1.0000
Epoch 21/100, Accuracy: 1.0000
Epoch 22/100, Accuracy: 1.0000
Epoch 23/100, Accuracy: 0.9667
Epoch 24/100, Accuracy: 1.0000
Epoch 25/100, Accuracy: 1.0000
Epoch 26/100, Accuracy: 1.0000
Epoch 27/100, Accuracy: 1.0000
Epoch 28/100, Accuracy: 1.0000
Epoch 29/100, Accuracy: 1.0000
Epoch 30/100, Accuracy: 1.0000
Epoch 31/100, Accuracy: 1.0000
Epoch 32/100, Accuracy: 1.0000
Epoch 33/100, Acc

In [7]:
from pytorch_tabular.models import SAINTModel

ImportError: cannot import name 'SAINTModel' from 'pytorch_tabular.models' (c:\Users\fadhil\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabular\models\__init__.py)

In [8]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, ModelConfig, OptimizerConfig, TrainerConfig

# Muat dataset Iris
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name='target')

# Gabungkan fitur dan target ke dalam satu DataFrame
data = pd.concat([X, y], axis=1)

# Bagi data menjadi train dan test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Konfigurasikan data
data_config = DataConfig(
    target=["target"],  # Nama kolom target
    continuous_cols=X.columns.tolist(),  # Nama kolom fitur numerik
)

# Konfigurasikan model
model_config = ModelConfig(
    task="classification",  # Tugas: "classification"
    model_name="saint",  # Nama model: gunakan string "saint" untuk SAINT
)

# Konfigurasikan optimisasi
optimizer_config = OptimizerConfig()

# Konfigurasikan pelatihan
trainer_config = TrainerConfig(
    auto_lr_find=True,
    max_epochs=100,
)

# Inisialisasi model dengan konfigurasi di atas
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

# Pelatihan model
tabular_model.fit(train=train_data)

# Evaluasi model
result = tabular_model.evaluate(test=test_data)
print(result)

# Prediksi menggunakan model terlatih
predictions = tabular_model.predict(test_data)
print(predictions.head())

TypeError: ModelConfig.__init__() got an unexpected keyword argument 'model_name'

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [6]:
class ObliviousDecisionTreeLayer(nn.Module):
    def __init__(self, input_dim, num_trees=10, tree_depth=6):
        super(ObliviousDecisionTreeLayer, self).__init__()
        self.num_trees = num_trees
        self.tree_depth = tree_depth
        self.weight = nn.Parameter(torch.randn(num_trees, tree_depth, input_dim))
        self.bias = nn.Parameter(torch.randn(num_trees, tree_depth))  # Bias berbentuk [num_trees, tree_depth]

    def forward(self, x):
        batch_size = x.size(0)
        outputs = []

        for i in range(self.num_trees):
            out = x
            for j in range(self.tree_depth):
                # Perluas 'bias' agar sesuai dengan dimensi batch
                bias = self.bias[i, j].unsqueeze(0).expand(batch_size, -1)  # Bias diperluas agar sesuai dengan batch_size
                # Operasikan linear layer dengan dimensi yang cocok
                decision = torch.sigmoid(F.linear(out, self.weight[i, j].unsqueeze(0), bias))
                # Sesuaikan dimensi 'out' untuk melanjutkan operasi
                out = decision * out
            outputs.append(out)

        # Rata-rata output dari semua pohon
        return torch.mean(torch.stack(outputs), dim=0)

# Definisi NODE Model
class NODE(nn.Module):
    def __init__(self, input_dim, output_dim, num_trees=10, tree_depth=6, hidden_dim=64):
        super(NODE, self).__init__()
        self.tree_layer = ObliviousDecisionTreeLayer(input_dim, num_trees=num_trees, tree_depth=tree_depth)
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # Sesuaikan input dimension sesuai dengan output dari tree_layer
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.tree_layer(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Persiapan data
data = load_iris()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Konversi ke tensor
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

# Inisialisasi model NODE
input_dim = X_train.shape[1]
output_dim = len(set(y))
model = NODE(input_dim, output_dim, num_trees=10, tree_depth=6, hidden_dim=64)

# Definisi optimizer dan loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluasi model
model.eval()
with torch.no_grad():
    y_pred_train = torch.argmax(model(X_train_t), dim=1)
    y_pred_test = torch.argmax(model(X_test_t), dim=1)

    train_acc = accuracy_score(y_train, y_pred_train.numpy())
    test_acc = accuracy_score(y_test, y_pred_test.numpy())
    f1 = f1_score(y_test, y_pred_test.numpy(), average='weighted')

    print(f'Train Accuracy: {train_acc:.4f}')
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'F1 Score: {f1:.4f}')

Epoch [10/100], Loss: 1.0929
Epoch [20/100], Loss: 1.0782
Epoch [30/100], Loss: 0.9923
Epoch [40/100], Loss: 0.6962
Epoch [50/100], Loss: 0.4592
Epoch [60/100], Loss: 0.2658
Epoch [70/100], Loss: 0.1123
Epoch [80/100], Loss: 0.0710
Epoch [90/100], Loss: 0.0583
Epoch [100/100], Loss: 0.0552
Train Accuracy: 0.9905
Test Accuracy: 1.0000
F1 Score: 1.0000


In [ ]:
class NODEModel(nn.Module):
    """
    Neural Oblivious Decision Ensembles (NODE) model.
    """
    def __init__(self, input_dim, num_trees, depth):
        """
        Initialize the NODE model.
        
        Args:
            input_dim (int): Number of input features.
            num_trees (int): Number of decision trees in the ensemble.
            depth (int): Depth of each decision tree.
        """
        super(NODEModel, self).__init__()
        self.num_trees = num_trees
        self.depth = depth
        
        # Create a list of decision trees
        self.trees = nn.ModuleList([nn.Sequential(
            nn.Linear(input_dim, 2 ** depth),
            nn.ReLU(),
            nn.Linear(2 ** depth, 1)
        ) for _ in range(num_trees)])